In [27]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import shutil

from google.cloud import storage

In [28]:
TEST_PATH = '/home/jupyter/test_images/'
TEST_L_PATH = '/home/jupyter/test_labels/'

FRAME_PATH = os.path.join(os.getcwd(), 'images') 

MASK_PATH = os.path.join(os.getcwd(), 'labels')

test_img_list_path = os.path.join(os.getcwd(), 'metadata/test_list.txt')

In [ ]:
storage_client = storage.Client(project = 'ih-final-project-277919')
bucket = storage_client.get_bucket('final_project_ih_data_dl')

In [29]:
def save_image(TEST_PATH, image_id):
    
    try:
        if os.path.isfile(os.path.join(TEST_PATH, "{}.jpg".format(image_id)))==True:
            return
        else:
            img_fp = os.path.join(FRAME_PATH, "{}.jpg".format(image_id))
            image = cv2.imread(img_fp)
            
            img_ts = os.path.join(TEST_PATH, "{}.jpg".format(image_id))

            cv2.imwrite(img_ts, image)
    except:
        #images missing that we won't add to the model
        return image_id

In [30]:
def save_label(TEST_L_PATH, image_id):
    
    shutil.copy2(f'{MASK_PATH}/{image_id}.txt', f'{TEST_L_PATH}/{image_id}.txt')


In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)


Save images and labels to file and also to the bucket

In [1]:
with open(test_img_list_path, "r") as test_img_list_f:

    for line in tqdm(test_img_list_f):

        image_id = line.strip().split('/')[-1].strip().split('.')[0]

        save_image(TEST_PATH, image_id)
        save_label(TEST_L_PATH, image_id)
        
        upload_blob(bucket,f'{TEST_PATH}{image_id}.jpg' ,f'images_test/{image_id}.jpg')
        upload_blob(bucket,f'{TEST_L_PATH}{image_id}.txt' ,f'labels_test/{image_id}.text')
        